In [ ]:
from Bio import SwissProt, SeqIO
import gzip

In [ ]:
cd ~/gpfs/projects/black_spruce/

In [ ]:
def get_accessions(dat_file):
    s = set()
    for rec in SwissProt.parse(open(dat_file)):
       [s.add(x) for x in rec.accessions]
    return s

In [ ]:
sprot_accessions = get_accessions("uniprot_sprot_plants.dat")

In [ ]:
len(sprot_accessions)

In [ ]:
def get_recs(accessions, fastagz):
    recs = []
    for rec in SeqIO.parse(gzip.open(fastagz), "fasta"):
        acc = rec.name.split("|")[1]
        if acc in accessions:
            recs.append(rec)
    return recs

def stream_recs_to_file(accessions, fastagz, out_file):
    with open(out_file, "w") as o:
        for rec in SeqIO.parse(gzip.open(fastagz), "fasta"):
            acc = rec.name.split("|")[1]
            if acc in accessions:
                SeqIO.write(rec, o, "fasta")

In [ ]:
sprot_recs = get_recs(sprot_accessions, "uniprot_sprot.fasta.gz")

In [ ]:
SeqIO.write(sprot_recs, open("uniprot_sprot_plants.fasta", "w"), "fasta")

```bash
/home/cfriedline/gpfs/src/ncbi-blast-2.2.30+/bin/makeblastdb \
-in uniprot_sprot_plants.fasta \
-dbtype prot \
-parse_seqids 
```

In [ ]:
trembl_accessions = get_accessions("uniprot_trembl_plants.dat")

In [ ]:
len(trembl_accessions)

In [ ]:
trembl_recs = stream_recs_to_file(trembl_accessions, 
                                  "uniprot_trembl.fasta.gz", 
                                  "uniprot_trembl_plants.fasta")

```bash
/home/cfriedline/gpfs/src/ncbi-blast-2.2.30+/bin/blastx \
-db uniprot_sprot_plants.fasta \
-query seqclean/all_ests.fa.clean_output/all_unigene_seq.fasta \
-out all_uniprot_sprot.xml \
-outfmt 5 \
-num_alignments 10 \
-evalue 1e-5 \
-num_threads 20
```


In [ ]:
from Bio import SearchIO
import pandas as pd

In [ ]:
cd ~/g/projects/black_spruce/

In [ ]:
dge_n = pd.read_csv("dge_N.csv", index_col=0)
dge_c = pd.read_csv("dge_C.csv", index_col=0)

In [ ]:
for qresult in SearchIO.parse("all_uniprot_sprot.xml", "blast-xml"):
    if qresult.id in dge_n.index:
        print qresult.id, len(qresult.hits)
        if qresult.id == "UN0195" or qresult.id == "UN0205":
            print qresult

In [ ]:
gene_contig_hits = pd.read_excel("gene_contig_hits.xls")

In [ ]:
gene_contig_hits.index = [x.replace("all_","") for x in gene_contig_hits.index]

In [ ]:
gene_contig_hits.head()

In [ ]:
c = gene_contig_hits.join(dge_c, how="inner")[["contig", "contig_start", "contig_end"]]
n = gene_contig_hits.join(dge_n, how="inner")[["contig", "contig_start", "contig_end"]]

In [ ]:
c['s'] = c.apply(lambda x: "%s [%d, %d]" % (x.contig, x.contig_start, x.contig_end), axis=1)
n['s'] = n.apply(lambda x: "%s [%d, %d]" % (x.contig, x.contig_start, x.contig_end), axis=1)

In [ ]:
pd.DataFrame(c.s)

In [ ]:
pd.DataFrame(n.s)